<h2> In-class transcript from Lecture 3, January 14, 2020</h2>

# Imports and defs for lecture

In [2]:
# These are the standard imports for CS 111. 
# This list may change as the quarter goes on.

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import sparse
from scipy import linalg as spla
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib tk

In [3]:
def Usolve(U, y):
    """Backward solve an upper triangular system Ux = y for x
    Parameters: 
      U: the matrix, must be square, upper triangular, with nonzeros on the diagonal
      y: the right-hand side vector
    Output:
      x: the solution vector to U @ x == y
    """
    
    raise NotImplemented("you will write Usolve() as part of homework 2")
        
    return x

In [4]:
def LUfactorNoPiv(A):
    """Factor a square matrix, A == L @ U (no partial pivoting)
    Parameters: 
      A: the matrix.
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
            
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    return (L, U)

In [5]:
def LUfactor(A, pivoting = True):
    """Factor a square matrix with partial pivoting, A[p,:] == L @ U
    Parameters: 
      A: the matrix.
      pivoting: whether or not to do partial pivoting
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
      p: the permutation vector that permutes the rows of A by partial pivoting
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Initialize p to be the identity permutation
    p = np.array(range(n))
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
        
        # Choose the pivot row and swap it into place
        if pivoting:
            piv_row = piv_col + np.argmax(LU[piv_col:, piv_col]) 
            assert LU[piv_row, piv_col] != 0., "can't find nonzero pivot, matrix is singular"
            LU[[piv_col, piv_row], :]  = LU[[piv_row, piv_col], :]
            p[[piv_col, piv_row]]      = p[[piv_row, piv_col]]
            
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    return (L, U, p)

In [42]:
def Lsolve(L, b):
    """Forward solve a unit lower triangular system Ly = b for y
    Parameters: 
      L: the matrix, must be square, lower triangular, with ones on the diagonal
      b: the right-hand side vector
    Output:
      y: the solution vector to L @ y == b
    """
    
    # Check the input
    m, n = L.shape
    assert m == n, "matrix L must be square"
    assert np.all(np.tril(L) == L), "matrix L must be lower triangular"
    assert np.all(np.diag(L) == 1), "matrix L must have ones on the diagonal"
    
    # Make a copy of b that we will transform into the solution
    y = b.astype(np.float64).copy()
    
    # Forward solve
    for col in range(n):
        y[col+1:] -= y[col] * L[col+1:, col]
        
    return y

# Lecture starts here

In [7]:
# Example of an upper triangular matrix

U = np.array([[2,7,1,8],[0,2,8,1],[0,0,8,2],[0,0,0,8]])
print(U)

[[2 7 1 8]
 [0 2 8 1]
 [0 0 8 2]
 [0 0 0 8]]


In [8]:
# Example of a unit lower triangular matrix

L = np.array([[1,0,0,0],[.5,1,0,0],[0,.5,1,0],[-.5,-.5,0,1]])
print(L)

[[ 1.   0.   0.   0. ]
 [ 0.5  1.   0.   0. ]
 [ 0.   0.5  1.   0. ]
 [-0.5 -0.5  0.   1. ]]


In [9]:
# Experiments with a random matrix

A = np.round(20*np.random.rand(5,5))
print(A)

[[ 6.  4. 18. 14. 12.]
 [10. 14.  2. 15.  0.]
 [17.  2. 20. 16.  2.]
 [ 6.  8. 19.  6.  5.]
 [18.  8. 11.  9.  5.]]


In [11]:
npla.matrix_rank(A)

5

In [12]:
B = A.copy()

In [13]:
B[:,1] = B[:,2] + B[:,3]

In [14]:
print(B)

[[ 6. 32. 18. 14. 12.]
 [10. 17.  2. 15.  0.]
 [17. 36. 20. 16.  2.]
 [ 6. 25. 19.  6.  5.]
 [18. 20. 11.  9.  5.]]


In [15]:
npla.matrix_rank(B)

4

In [16]:
npla.norm(A)

57.7148161220323

In [17]:
A

array([[ 6.,  4., 18., 14., 12.],
       [10., 14.,  2., 15.,  0.],
       [17.,  2., 20., 16.,  2.],
       [ 6.,  8., 19.,  6.,  5.],
       [18.,  8., 11.,  9.,  5.]])

In [18]:
npla.cond(A)

8.456986674625055

In [19]:
npla.cond(B)

1.9098658445932554e+17

In [20]:
A

array([[ 6.,  4., 18., 14., 12.],
       [10., 14.,  2., 15.,  0.],
       [17.,  2., 20., 16.,  2.],
       [ 6.,  8., 19.,  6.,  5.],
       [18.,  8., 11.,  9.,  5.]])

In [21]:
# Creating a right-hand side for which we know the answer to Ax=b

xorig = np.round(10*np.random.rand(5))
print("original x:", xorig)
b = A @ xorig
print("right-hand side b:", b)

original x: [ 2.  1. 10.  4.  7.]
right-hand side b: [336. 114. 314. 269. 225.]


In [22]:
x = npla.solve(A,b)

In [23]:
print("computed x", x)

computed x [ 2.  1. 10.  4.  7.]


In [24]:
x

array([ 2.,  1., 10.,  4.,  7.])

In [25]:
x - xorig

array([-4.44089210e-16, -2.55351296e-15,  0.00000000e+00,  1.77635684e-15,
       -1.77635684e-15])

In [26]:
npla.norm(x-xorig)

3.609505629278167e-15

In [27]:
# Now a different matrix A and right-hand side vector b

A = np.array([[ 2. ,  7. ,  1. ,  8. ],
       [ 1. ,  5.5,  8.5,  5. ],
       [ 0. ,  1. , 12. ,  2.5],
       [-1. , -4.5, -4.5,  3.5]])
print("A:", A,'\n')
b = np.array([17. ,  2.5, -7. , 10.5])
print("b:", b)

A: [[ 2.   7.   1.   8. ]
 [ 1.   5.5  8.5  5. ]
 [ 0.   1.  12.   2.5]
 [-1.  -4.5 -4.5  3.5]] 

b: [17.   2.5 -7.  10.5]


In [28]:
L

array([[ 1. ,  0. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. ,  0. ],
       [ 0. ,  0.5,  1. ,  0. ],
       [-0.5, -0.5,  0. ,  1. ]])

In [29]:
U

array([[2, 7, 1, 8],
       [0, 2, 8, 1],
       [0, 0, 8, 2],
       [0, 0, 0, 8]])

In [30]:
A


array([[ 2. ,  7. ,  1. ,  8. ],
       [ 1. ,  5.5,  8.5,  5. ],
       [ 0. ,  1. , 12. ,  2.5],
       [-1. , -4.5, -4.5,  3.5]])

In [31]:
L @ U

array([[ 2. ,  7. ,  1. ,  8. ],
       [ 1. ,  5.5,  8.5,  5. ],
       [ 0. ,  1. , 12. ,  2.5],
       [-1. , -4.5, -4.5,  3.5]])

In [32]:
B

array([[ 6., 32., 18., 14., 12.],
       [10., 17.,  2., 15.,  0.],
       [17., 36., 20., 16.,  2.],
       [ 6., 25., 19.,  6.,  5.],
       [18., 20., 11.,  9.,  5.]])

In [33]:
b

array([17. ,  2.5, -7. , 10.5])

In [46]:
Lsolve?

In [35]:
y = Lsolve(L,b)
y

array([17., -6., -4., 16.])

In [36]:
x = Usolve(U,y)
x

array([ 1.,  0., -1.,  2.])

In [37]:
A @ x

array([17. ,  2.5, -7. , 10.5])

In [38]:
A @ x - b

array([0., 0., 0., 0.])